In [1]:
import numpy as np
import pandas as pd
from scipy import stats

import pylab as plt
import seaborn as sns

from importlib import reload
import os
import re

import sys
sys.path.insert(1,'../scripts')
import utils
import plots
import predict_regress

# Load Data

In [3]:
demo = pd.read_csv('/scratch/c.c21013066/data/ppmi/phenotypes2021/demographics_clean.csv',parse_dates=['date_birth','date_diagnosis'])
grouped_mean = pd.read_csv('/scratch/c.c21013066/data/ppmi/accelerometer/weekly_mean.csv',parse_dates=['date_y'])
grouped_mean = pd.merge(demo,grouped_mean,on='participant',how='right')
grouped_mean = grouped_mean[grouped_mean['diagnosis']=='pd']
grouped_mean = grouped_mean.groupby('participant').last()
grouped_mean['diagnosis_age'] = (grouped_mean['date_diagnosis'] - grouped_mean['date_birth'] ) / np.timedelta64(1,'Y')
grouped_mean['time_since_diagnosis'] = grouped_mean['visit_age']-grouped_mean['diagnosis_age']
grouped_mean['male'] = grouped_mean['gender'].replace(['f','m'],[0,1])

In [4]:
neuropsychiatric = ['stai_trait','stai_state','gds','quip']
cognition = ['semantic_fluency','moca','benton','lns','hvlt_recall','hvlt_recognition','hvlt_retention','symbol_digit']
autonome = ['epworth','rbd','systolic_bp_drop','scopa_aut']
daily = ['se_adl','updrs_i']
motor = ['updrs_ii','updrs_iii_OFF','updrs_iii_ON','updrs_iv']
medication = ['LEDD']
targets = np.hstack([motor,cognition,neuropsychiatric,autonome,daily,medication])
covs = ['diagnosis_age','time_since_diagnosis','male']
predictors = grouped_mean.filter(regex='(walking|step|efficiency|total_sleep|pulse|deep|light|rem|nrem|rmssd|wake)').columns
sleep_col = grouped_mean.filter(regex='(efficiency|total_sleep|deep|light|rem|nrem|wake)').columns
phys = grouped_mean.filter(regex='(walking|step)').columns
vital = grouped_mean.filter(regex='(pulse|rmssd)').columns
import re
predictors_filt = [a for a in predictors if not re.search('_ms', a)]
sleep_col = [a for a in sleep_col if not re.search('_ms', a)]
phys = [a for a in phys if not re.search('_ms', a)]
vital = [a for a in vital if not re.search('_ms', a)]

# Run Regression

In [16]:
reload(predict_regress)
reload(plots)

for target in targets:
    clean = grouped_mean.dropna(subset=np.hstack([target,covs]))
    # remove any predictors where more than 5% missing
    missing_percentages = clean[predictors_filt].isnull().sum() / len(clean) * 100
    columns_to_remove = missing_percentages[missing_percentages > 5].index
    print(columns_to_remove)
    preds = np.setdiff1d(predictors_filt,columns_to_remove)
    print(preds)
    clean = clean.drop(columns=columns_to_remove)
    clean = clean.dropna(subset=preds)
    clean = clean.reset_index(drop=True)
    folder_path = f'/scratch/c.c21013066/data/ppmi/analyses/predictclinical/{target}/'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    else:
        print(f"Folder '{folder_path}' already exists.")
    print(clean.shape)
    coefs,cl = predict_regress.run_regression(clean,preds,covs,target,
                                           save=folder_path)

Index(['sleep_efficiency', 'num_awakenings', 'total_sleep_time_hour',
       'wake_after_sleep_onset_hour', 'total_nrem_time_hour',
       'total_rem_time_hour', 'total_deep_nrem_time_hour',
       'total_light_nrem_time_hour'],
      dtype='object')
['hourly_mean_pulse_rate' 'hourly_mean_rmssd' 'hourly_median_rmssd'
 'hourly_rmssd_variance' 'hourly_step_count_sum' 'sum_walking_minutes']
Folder '/scratch/c.c21013066/data/ppmi/analyses/predictclinical/updrs_ii/' already exists.
(83, 59)
failed plotting for  updrs_ii
Index(['sleep_efficiency', 'num_awakenings', 'total_sleep_time_hour',
       'wake_after_sleep_onset_hour', 'total_nrem_time_hour',
       'total_rem_time_hour', 'total_deep_nrem_time_hour',
       'total_light_nrem_time_hour'],
      dtype='object')
['hourly_mean_pulse_rate' 'hourly_mean_rmssd' 'hourly_median_rmssd'
 'hourly_rmssd_variance' 'hourly_step_count_sum' 'sum_walking_minutes']
Folder '/scratch/c.c21013066/data/ppmi/analyses/predictclinical/updrs_iii_OFF/' already 

../scripts/plots.py:31: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10,10))


failed plotting for  se_adl
Index(['sleep_efficiency', 'num_awakenings', 'total_sleep_time_hour',
       'wake_after_sleep_onset_hour', 'total_nrem_time_hour',
       'total_rem_time_hour', 'total_deep_nrem_time_hour',
       'total_light_nrem_time_hour'],
      dtype='object')
['hourly_mean_pulse_rate' 'hourly_mean_rmssd' 'hourly_median_rmssd'
 'hourly_rmssd_variance' 'hourly_step_count_sum' 'sum_walking_minutes']
Folder '/scratch/c.c21013066/data/ppmi/analyses/predictclinical/updrs_i/' already exists.
(83, 59)
failed plotting for  updrs_i
Index(['sleep_efficiency', 'num_awakenings', 'total_sleep_time_hour',
       'wake_after_sleep_onset_hour', 'total_nrem_time_hour',
       'total_rem_time_hour', 'total_deep_nrem_time_hour',
       'total_light_nrem_time_hour'],
      dtype='object')
['hourly_mean_pulse_rate' 'hourly_mean_rmssd' 'hourly_median_rmssd'
 'hourly_rmssd_variance' 'hourly_step_count_sum' 'sum_walking_minutes']
Folder '/scratch/c.c21013066/data/ppmi/analyses/predictclinica

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

...

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>